In [11]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.serialization import load_pem_public_key
from cryptography.hazmat.primitives.asymmetric.padding import OAEP, MGF1
from cryptography.hazmat.primitives import serialization
from cryptography.fernet import Fernet

# ----------------------- Server -------------------------
def generate_rsa_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537, key_size=2048, backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

def encrypt_message(symmetric_key, message):
    if symmetric_key:
        return Fernet(symmetric_key).encrypt(message)
    else:
        raise Exception("Symmetric key not set")



def generate_symmetric_key():
    return Fernet.generate_key()


def encrypt_msg_with_public_key(msg, public_key):
    encrypted_msg = public_key.encrypt(
        msg,
        OAEP(
            mgf=MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None
        ),
    )
    return encrypted_msg


def decrypt_msg_with_private_key(encrypted_msg, private_key):
    decrypted_msg = private_key.decrypt(
        encrypted_msg,
        OAEP(
            mgf=MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None
        ),
    )
    return decrypted_msg

def symmetric_encrypt_msg(msg, key):
    f = Fernet(key)
    return f.encrypt(msg)

def symmetric_decrypt_msg(encrypted_msg, key):
    f = Fernet(key)
    return f.decrypt(encrypted_msg)


def decrypt_symmetric_key(private_key, encrypted_key):
    decrypted_key = private_key.decrypt(
        encrypted_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None,
        ),
    )
    return decrypted_key


def decrypt_msg(symmetric_key, encrypted_msg):
    if symmetric_key:
        return Fernet(symmetric_key).decrypt(encrypted_msg)
    else:
        return None


# --------------------------- Client ----------------------
def encrypt_msg_with_public_key(msg, public_key_pem):
    public_key = load_pem_public_key(public_key_pem, backend=default_backend())
    encrypted_msg = public_key.encrypt(
        msg,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None,
        ),
    )
    return encrypted_msg





In [13]:
import requests
from cryptography.hazmat.primitives import serialization

def get_public_key_pem(public_key):
    return public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

session = requests.Session()

# 请求服务器的公钥
response = session.get("http://localhost:8080/public_key")
server_public_key = serialization.load_pem_public_key(response.content)

# 生成对称密钥并加密
symmetric_key = generate_symmetric_key()

# 使用公钥对象获取 PEM 格式的公钥
server_public_key_pem = get_public_key_pem(server_public_key)

# 使用 PEM 格式的公钥进行加密
encrypted_symmetric_key = encrypt_msg_with_public_key(symmetric_key, server_public_key_pem)

# 发送加密的对称密钥给服务器
session.post("http://localhost:8080/receive_symmetric_key", data=encrypted_symmetric_key)

# 使用对称密钥加密通信内容
data = b"Secret message"
encrypted_data = symmetric_encrypt_msg(data, symmetric_key)
# 发送加密数据
response = session.post("http://localhost:8080/encrypted_endpoint", data=encrypted_data)

# 解密返回的数据
encrypted_response = response.content
decrypted_response = symmetric_decrypt_msg(encrypted_response, symmetric_key)

# 处理或打印解密后的响应
print("Origin request:", data)
print("Encrypted request:", encrypted_data)
print("Origin response:", encrypted_response)
print("Decrypted response:", decrypted_response.decode())


Origin request: b'Secret message'
Encrypted request: b'gAAAAABlktMxqj7ViMpOXKHREv1DtZ0ylSGdQonmpRKhcYPk60UMwUZy0aXGSV0kaL-0x_hNzrAU6gE0mKC9fzuE1s0NJPFwxQ=='
Origin response: b'gAAAAABlktMxaMF6K9OWdBZhtWrN0vye_ICNr0gaJsmvsoy5_H-ceCNXx2T7a9UNJr1Xm4BMZtaLx6yQJskYkQvq8wkaVHu1DIfvWVjqZcJz6stQ-j40uXM='
Decrypted response: Response to encrypted request
